In [106]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

In [107]:
import matplotlib.pyplot as plt

In [108]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [109]:
x_train = np.reshape(x_train, (-1, 784))
y_train = np.reshape(y_train, (60000, -1))
x_train = np.hstack((x_train, y_train))
x_test = np.reshape(x_test, (-1, 784))
y_test = np.reshape(y_test, (10000, -1))
x_test = np.hstack((x_test, y_test))
x_test.shape


(10000, 785)

In [ ]:
plt.title(y_train[10])
plt.imshow(x_train[10], cmap=plt.get_cmap('gray_r'))
plt.show()

In [ ]:
def display_digit(num):
    label = y_train[num]
    image = x_train[num]
    plt.title('Example: {}  Label: {}'.format(num, label))
    plt.imshow(image, cmap=plt.get_cmap('gray_r'))
    plt.show()

In [ ]:
display_digit(2908)

In [111]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(785, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.0),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [112]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

In [113]:
model.fit(XX_train, y_train, epochs=50)

Epoch 1/50
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1129 - accuracy: 0.9658
Epoch 2/50
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0310 - accuracy: 0.9899
Epoch 3/50
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0190 - accuracy: 0.9940
Epoch 4/50
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0151 - accuracy: 0.9952
Epoch 5/50
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0101 - accuracy: 0.9966
Epoch 6/50
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0085 - accuracy: 0.9974
Epoch 7/50
1875/1875 [==============================] - 7s 3ms/step - loss: 0.0076 - accuracy: 0.9974
Epoch 8/50
1875/1875 [==============================] - 7s 3ms/step - loss: 0.0087 - accuracy: 0.9973
Epoch 9/50
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0041 - accuracy: 0.9987
Epoch 10/50
1875/1875 [==============================] - 7s 4ms/step - loss: 0.008

In [114]:
loss, acc = model.evaluate(x_test, y_test)
print("Loss = {}, accuracy = {}".format(loss, acc))

313/313 [==============================] - 0s 1ms/step - loss: 0.0634 - accuracy: 0.9946
Loss = 0.06342767179012299, accuracy = 0.9945999979972839


In [115]:
loss, acc = model.evaluate(XX_train, y_train)
print("Loss = {}, accuracy = {}".format(loss, acc))

1875/1875 [==============================] - 3s 2ms/step - loss: 8.9291e-04 - accuracy: 0.9998
Loss = 0.0008929140167310834, accuracy = 0.999750018119812


In [ ]:
predictions = model.predict(x_test[0:1,:,:])
print(predictions)
print(y_test[0])